In [1]:
from sys import path as syspath
from os import path as ospath
import os, glob
from joblib import Parallel, delayed
from timeit import default_timer as timer
import numpy as np
import matplotlib.pyplot as plt
import inspect
import json
import multiprocessing
import functools


#CBSA
syspath.append(ospath.join(ospath.expanduser("~"), 'CBSA'))
from cbsa import ReactionSystem

#StochPy
import stochpy as sp

#GillesPy
import gillespy2 as glp
from gillespy2.solvers.numpy.basic_ode_solver import BasicODESolver
from gillespy2.solvers.numpy.basic_tau_leaping_solver import BasicTauLeapingSolver
from gillespy2.solvers.numpy.ssa_solver import NumPySSASolver

#STEPS
import steps.geom as swm
import steps.model as smodel
import steps.rng as srng
import steps.solver as ssolver


#MAX COMPUTING TIME
MAX_COMP_TIME = 60


#######################################################################
#                                                                     #
#            Welcome to the interactive StochPy environment           #
#                                                                     #
#######################################################################
#  StochPy: Stochastic modeling in Python                             #
#  http://stochpy.sourceforge.net                                     #
#  Copyright(C) T.R Maarleveld, B.G. Olivier, F.J Bruggeman 2010-2015 #
#  DOI: 10.1371/journal.pone.0079345                                  #
#  Email: tmd200@users.sourceforge.net                                #
#  VU University, Amsterdam, Netherlands                              #
#  Centrum Wiskunde Informatica, Amsterdam, Netherlands               #
#  StochPy is distributed under the BSD licence.                      #
###############################################################

In [2]:
source_file_DF = inspect.getsourcefile(sp.SSA)
print(source_file_DF)

/home/burke/.local/lib/python3.7/site-packages/stochpy/modules/StochSim.py


In [3]:
def with_timeout(timeout):
    def decorator(decorated):
        @functools.wraps(decorated)
        def inner(*args, **kwargs):
            pool = multiprocessing.pool.ThreadPool(1)
            async_result = pool.apply_async(decorated, args, kwargs)
            try:
                return async_result.get(timeout)
            except multiprocessing.TimeoutError:
                return "timeout "+str(timeout)
        return inner
    return decorator

In [7]:
def mean_value(data,index):
    time_steps = np.diff(data[:,0])
    sum_ts = np.sum(time_steps)
    if not sum_ts: return 0.
    return np.sum(time_steps*data[:,index][0:-1])/sum_ts

def cbsa2stochpy(cbsa_model,path="/home/burke/Stochpy/pscmodels/"):
    
    model_str = ""
    for i in range(1,cbsa_model.exp_n_reactions):
        reactants = np.where(cbsa_model.expS[:,i] < 0)[0]
        reactants_sto = list(cbsa_model.expS[:,i][reactants]*-1)
        modifiers = np.where(cbsa_model.expR[:,i] > 0)[0]
        modifiers_sto = list(cbsa_model.expR[:,i][modifiers])
        products = np.where(cbsa_model.expS[:,i] > 0)[0]
        products_sto = list(cbsa_model.expS[:,i][products])

        psc_reactants = []
        psc_modifiers = []
        psc_products = []
        
        if len(reactants):
            psc_reactants = ["{"+str(reactants_sto[j])+"}M"+str(reactants[j]) for j in range(len(reactants))]
        if len(modifiers):
            psc_modifiers = ["{"+str(modifiers_sto[j])+"}M"+str(modifiers[j]) for j in range(len(modifiers))]
        if len(products):
            psc_products = ["{"+str(products_sto[j])+"}M"+str(products[j]) for j in range(len(products))]
        
        psc_reactants += psc_modifiers
        psc_products += psc_modifiers
        
        if not len(psc_reactants):
            psc_reactants = ['$pool']
            
        if not len(psc_products):
            psc_products = ['$pool']
        
        k_mols = [["M"+str(reactants[j]) for k in range(reactants_sto[j])] for j in range(len(reactants))]
        k_mols += [["M"+str(modifiers[j]) for k in range(modifiers_sto[j])] for j in range(len(modifiers))]
        k_mols = [item for sublist in k_mols for item in sublist]
        
        model_str += "R"+str(i)+":\n\t"
        model_str += "+".join(psc_reactants)            
        model_str += " > "
        model_str += "+".join(psc_products)        
        model_str += "\n\t"
        model_str += "*".join(k_mols+["k"+str(i)])
        model_str += "\n\n"
    
    for i in range(1,cbsa_model.exp_n_reactions):
        model_str += "k"+str(i)+" = "+str(cbsa_model.exp_k[i])+"\n"
    
    model_str += "\n"
    
    for i in range(1,cbsa_model.exp_n_molecules):
        model_str += "M"+str(i)+" = "+str(int(cbsa_model.exp_x0[i]))+"\n"
    
    if not os.path.exists(path):
        os.makedirs(path)
    
    model_file = "model_tmp"+str(timer())+".psc"
    
    with open(path+model_file,"w") as f:
        f.write(model_str)

    smod = sp.SSA()
    smod.Model(model_file)
    
    return smod


def cbsa2gillespy(cbsa_model):
    
    gilles = glp.Model(name="Model")

    k = [glp.Parameter(name='k'+str(i), expression=cbsa_model.exp_k[i]) for i in range(1,cbsa_model.exp_n_reactions)]
    gilles.add_parameter(k)
    mols = [glp.Species(name='M'+str(i), initial_value=int(cbsa_model.exp_x0[i])) for i in range(1,cbsa_model.exp_n_molecules)]
    gilles.add_species(mols)
    
    reactions = []
    
    for i in range(1,cbsa_model.exp_n_reactions):
        reactants = list(np.where(cbsa_model.expS[:,i] < 0)[0])
        reactants_sto = list(cbsa_model.expS[:,i][reactants]*-1)
        modifiers = list(np.where(cbsa_model.expR[:,i] > 0)[0])
        modifiers_sto = list(cbsa_model.expR[:,i][modifiers])
        products = list(np.where(cbsa_model.expS[:,i] > 0)[0])
        products_sto = list(cbsa_model.expS[:,i][products])
        
        reactants += modifiers
        reactants_sto += modifiers_sto
        products += modifiers
        products_sto += modifiers_sto
        
        reactions.append(glp.Reaction(name="R"+str(i),
                                      rate=k[i-1],
                                      reactants={mols[reactants[j]-1]:reactants_sto[j] for j in range(len(reactants))},
                                      products={mols[products[j]-1]:products_sto[j] for j in range(len(products))}
                                     )
                        )
    
    gilles.add_reaction(reactions)
    
    return gilles

def cbsa2steps(cbsa_model):
    
    mdl = smodel.Model()
    vsys = smodel.Volsys('vsys', mdl)    
    mols = [smodel.Spec('M'+str(i), mdl) for i in range(1,cbsa_model.exp_n_molecules)]    
    reactions = []    
    for i in range(1,cbsa_model.exp_n_reactions):
        reactants = list(np.where(cbsa_model.expS[:,i] < 0)[0])
        reactants_sto = list(cbsa_model.expS[:,i][reactants]*-1)
        modifiers = list(np.where(cbsa_model.expR[:,i] > 0)[0])
        modifiers_sto = list(cbsa_model.expR[:,i][modifiers])
        products = list(np.where(cbsa_model.expS[:,i] > 0)[0])
        products_sto = list(cbsa_model.expS[:,i][products])
        
        reactants += modifiers
        reactants_sto += modifiers_sto
        products += modifiers
        products_sto += modifiers_sto
        
        reactants_objs = [[mols[reactants[j]-1] for k in range(reactants_sto[j])] for j in range(len(reactants))]
        reactants_objs = [item for sublist in reactants_objs for item in sublist]
        
        products_objs = [[mols[products[j]-1] for k in range(products_sto[j])] for j in range(len(products))]
        products_objs = [item for sublist in products_objs for item in sublist]
        
        reactions.append(smodel.Reac("R"+str(i), vsys, lhs=reactants_objs, rhs=products_objs, kcst=cbsa_model.exp_k[i]))
    
    wmgeom = swm.Geom()

    comp = swm.Comp('comp', wmgeom)
    comp.addVolsys('vsys')
    comp.setVol(1.6667e-21)

    r = srng.create('mt19937', 256)
    r.initialize(int(timer()))
    sim = ssolver.Wmdirect(mdl, wmgeom, r)
    sim.reset()

    for i in range(1,cbsa_model.exp_n_molecules):
        sim.setCompConc('comp', 'M'+str(i), cbsa_model.exp_x0[i]*1e-6)
    
    return sim
    
def clear_tmp_psc(path="/home/burke/Stochpy/pscmodels/"):
    for fl in glob.glob(path+"model_tmp*"):
        os.remove(fl)

def generate_cbsa_diffusion_model(sqrt_n_spaces,init_mols,diffusion_k,max_dt=0.1,total_sim_time=100):
    
    S = [[0]]
    R = [[0]]

    x = [init_mols]
    k = [0.]
    diff_k = [diffusion_k]    
    
    cbsa = ReactionSystem(S,R)
    if sqrt_n_spaces<2:
        cbsa.setup()
    else:
        cbsa.setup(sqrt_n_spaces**2,'toroid')

    cbsa.set_x(x)
    cbsa.set_k(k,diff_k)
    
    cbsa.bench_max_dt = max_dt
    cbsa.bench_total_sim_time = total_sim_time
    
    return cbsa

def generate_cbsa_burst_model(sqrt_n_spaces=1,diffusion_k=0.1,max_dt=0.1,total_sim_time=100):    
    
    S = [[-1,1,0,0],
         [1,-1,0,0],
         [0,0,1,-1]]

    R = [[0,0,1,0],
         [0,0,0,0],
         [0,0,0,0]]

    x = [0,1,0]
    k = [0.05,0.05,200,0.5]
    diff_k = [diffusion_k for i in range(len(x))]

    cbsa = ReactionSystem(S,R)
    if sqrt_n_spaces<2:
        cbsa.setup()
    else:
        cbsa.setup(sqrt_n_spaces**2,'toroid')
    cbsa.set_x(x,mol_per_subspace=True)
    cbsa.set_k(k,diff_k)
    
    cbsa.bench_max_dt = max_dt
    cbsa.bench_total_sim_time = total_sim_time
    
    return cbsa

@with_timeout(MAX_COMP_TIME)
def run_cbsa(cbsa_model,use_opencl=False,output="time"):
    cbsa_model.setup_simulation(use_opencl=use_opencl,alpha=0.5,max_dt=cbsa_model.bench_max_dt)
    start = timer()
    cbsa_model.compute_simulation(cbsa_model.bench_total_sim_time,batch_steps=1)
    time_elapsed = timer() - start
    cbsa_data = cbsa_model.simulation_data
    
    return get_results(time_elapsed,cbsa_data,output)

def run_cbsa_gpu(cbsa_model,output="time"):
    return run_cbsa(cbsa_model,use_opencl=True,output=output)

@with_timeout(MAX_COMP_TIME)
def run_stochpy(cbsa_model,method="Direct",output="time"):
    smod = cbsa2stochpy(cbsa_model)
    start = timer()
    smod.DoStochSim(method=method,mode='time',end=cbsa_model.bench_total_sim_time,trajectories=1)
    time_elapsed = timer() - start
    sp_ssa_data = smod.data_stochsim.getSpecies()
    return get_results(time_elapsed,sp_ssa_data,output)

def run_stochpy_ssa(cbsa_model,output="time"):
    return run_stochpy(cbsa_model,output=output)

def run_stochpy_tauleap(cbsa_model,output="time"):
    return run_stochpy(cbsa_model,method="Tauleap",output=output)

@with_timeout(MAX_COMP_TIME)
def run_gillespy(cbsa_model,method="Direct",output="time"):
    methods = {"Direct":NumPySSASolver,"Tauleap":BasicTauLeapingSolver,"ODE":BasicODESolver}
    gilles = cbsa2gillespy(cbsa_model)
    gilles.timespan(np.cumsum(np.array([cbsa_model.bench_max_dt for i in range(int(cbsa_model.bench_total_sim_time/cbsa_model.bench_max_dt))])))
    start = timer()
    result = gilles.run(solver = methods[method],number_of_trajectories=1)
    time_elapsed = timer() - start
    gilles_ssa_data = np.column_stack([result['time']]+[result['M'+str(i)] for i in range(1,cbsa_model.exp_n_molecules)])
    return get_results(time_elapsed,gilles_ssa_data,output)
    
def run_gillespy_ssa(cbsa_model,output="time"):
    return run_gillespy(cbsa_model,method="Direct",output=output)

def run_gillespy_tauleap(cbsa_model,output="time"):
    return run_gillespy(cbsa_model,method="Tauleap",output=output)

def run_gillespy_ode(cbsa_model,output="time"):
    return run_gillespy(cbsa_model,method="ODE",output=output)

@with_timeout(MAX_COMP_TIME)
def run_steps(cbsa_model,output="time"):
    sim = cbsa2steps(cbsa_model)
    
    tpnt = np.arange(0.0, cbsa_model.bench_total_sim_time+cbsa_model.bench_max_dt, cbsa_model.bench_max_dt)
    total_steps = tpnt.shape[0]
    res = np.zeros([total_steps, cbsa_model.exp_n_molecules-1])
    start = timer()
    for t in range(0,total_steps):
        sim.run(tpnt[t])
        for i in range(1,cbsa_model.exp_n_molecules):
            res[t,i-1] = sim.getCompCount('comp', 'M'+str(i))
    time_elapsed = timer() - start
    steps_data = np.zeros((res.shape[0],res.shape[1]+1))
    steps_data[:,0] = tpnt
    steps_data[:,1:] = res
    return get_results(time_elapsed,steps_data,output)

def get_results(time,data,output="time-last value"):
    data = np.array(data)
    if output == "time-last value":
        return [time]+list(np.round(data[-1,1:],0))
    elif output == "time-mean":
        return [time]+[mean_value(data,i) for i in range(1,data.shape[1])]
    elif output=="simdata":
        return np.array(data).tolist()
    elif output=="time":
        return time
        

In [8]:
simulation_info = {
    "Experiment ID":None,
    "Experiment Type":None,
    "Model Name":None,
    "Model Generator Function":None,
    "Model Generator Parameters": None,
    "Simulation Method":None,
    "Simulation Method Function":None,
    "Output Type":None,    
    "Replicates":None,
    "Result":None
}

def run_simulation(info,output_file):
    exp_id = info["Experiment ID"]
    model_generator = info["model_gen"]
    params = info["Model Generator Parameters"]
    #method = globals()[info["Simulation Method Function"]]
    method = info["sim_func"]
    replicates = info["Replicates"]
    output_type = info["Output Type"]
    model = model_generator(**params)
    info["Result"] = [method(model,output=output_type) for i in range(replicates)]
    del info["sim_func"]
    del info["model_gen"]
    with open(output_file,"a+") as f:
        f.write("#ExperimentID "+str(exp_id)+"\n")
        f.write(json.dumps(info))
        f.write("\n")
    return exp_id

def run_benchmark(infos_file,n_jobs=10,output_raw_file="benchmark.data",output_json_file="benchmark_result.json",verbose=False):
    
    if os.path.exists(output_raw_file):
        os.remove(output_raw_file)
        
    with open(infos_file) as f:
        infos = json.load(f)
        
    for i in range(len(infos)):
        infos[i]["sim_func"] = globals()[infos[i]["Simulation Method Function"]]
        infos[i]["model_gen"] = globals()[infos[i]["Model Generator Function"]]
        
    Parallel(n_jobs=n_jobs)(delayed(run_simulation)(i,output_raw_file) for i in infos)
    
    result = {}
    with open(output_raw_file) as f:
        exp_id = None
        for line in f.readlines():
            if line[0] == "#":
                exp_id = line.split(" ")[1].replace("\n","")
            else:
                if exp_id in result.keys():
                    print("WARNING: duplicated Experiment ID: "+exp_id)
                result[exp_id] = json.loads(line)
                
    with open(output_json_file,"w") as f:
        json.dump(result,f)

    if os.path.exists(output_raw_file):
        os.remove(output_raw_file)
    clear_tmp_psc()
    
    if verbose:
        print("Finished")
            
def save_experiment_infos(infos,file="experiment_infos.json"):
    with open(file,"w") as f:
        json.dump(infos,f)

In [9]:
diff_parameters = {
    "sqrt_n_spaces":4,
    "init_mols":100,
    "diffusion_k":0.1,
    "max_dt":0.1,
    "total_sim_time":1000
}

simulation_info = {
    "Experiment ID":1,
    "Experiment Type":"Number of Reactions",
    "Model Name":"Diffusion Torus",
    "Model Generator Function":"generate_cbsa_diffusion_model",
    "Model Generator Parameters": diff_parameters.copy(),
    "Simulation Method":"CBSA",
    "Simulation Method Function":"run_cbsa",
    "Output Type":"time-last value",
    "Replicates":10  
}
infos = [simulation_info.copy() for i in range(5)]

for i in range(len(infos)):
    infos[i]["Experiment ID"] = i
#print(infos)
#Parallel(n_jobs=10)(delayed(run_simulation)(i) for i in infos)
#run_simulation(simulation_info,output_file="benchmark.data")
save_experiment_infos(infos,"experiment_infos.json")
run_benchmark("experiment_infos.json",n_jobs=10,output_raw_file="benchmark.data",output_json_file="benchmark_result.json",verbose=True)

Finished


In [10]:
max_dt = 0.1
total_sim_time = 10
cbsa_model = generate_cbsa_diffusion_model(4,100,0.1,max_dt,total_sim_time)
print(run_cbsa(cbsa_model))
print(run_cbsa_gpu(cbsa_model))
print(run_stochpy_ssa(cbsa_model))
print(run_gillespy_tauleap(cbsa_model))
print(run_steps(cbsa_model))
clear_tmp_psc()

0.03442802606150508
0.1271713130408898
0.03568946896120906
0.12677977699786425
0.004879063926637173
